In [ ]:
class D_R(nn.Module):
    def __init__(self, input_dim, input_len, total_level, current_level, dropout, enc_hidden, output_len, K_IMP):
        super().__init__()
        self.current_level = current_level
        self.input_len = input_len
        self.input_dim = input_dim
        self.enc_hidden = enc_hidden
        self.output_len = output_len
        self.dropout = dropout
        self.K_IMP = K_IMP
        self.total_level = total_level

        self.MCD = MCD(K_IMP, kernel_size=(1, 3), soft_max=False)

        if current_level == 0:
            pass
        else:
            self.branch_slelect = nn.Sequential(
                nn.Linear(input_len, enc_hidden),
                nn.BatchNorm2d(enc_hidden, affine=True),
                nn.LeakyReLU(),
                nn.Linear(enc_hidden, 2),
            )

            self.reconstruct_proj_left = nn.Sequential(
                nn.Linear(input_len, enc_hidden),
                nn.BatchNorm1d(enc_hidden, affine=True),
                nn.LeakyReLU(),
                nn.Dropout(self.dropout),
                nn.Linear(enc_hidden, input_len),
                nn.BatchNorm1d(enc_hidden, affine=True),
                nn.LeakyReLU()
            )

            self.reconstruct_proj_right = nn.Sequential(
                nn.Linear(input_len, enc_hidden),
                nn.BatchNorm1d(enc_hidden, affine=True),
                nn.LeakyReLU(),
                nn.Dropout(self.dropout),
                nn.Linear(enc_hidden, input_len),
                nn.BatchNorm1d(enc_hidden, affine=True),
                nn.LeakyReLU()
            )

            self.EMDNet_Tree_left = D_R(
                input_dim, input_len, total_level, current_level - 1, dropout, enc_hidden, output_len, K_IMP)
            self.EMDNet_Tree_right = D_R(
                input_dim, input_len, total_level, current_level - 1, dropout, enc_hidden, output_len, K_IMP)

            if current_level == total_level:
                self.forecast_proj1 = nn.ModuleList(
                    [nn.Linear(self.input_len, self.enc_hidden)
                     for i in range(2 ** total_level * self.K_IMP)]
                )
                self.activate = nn.LeakyReLU()
                self.forecast_proj2 = nn.ModuleList(
                    [nn.Linear(self.enc_hidden, self.input_len)
                     for i in range(2 ** total_level * self.K_IMP)]
                )
            else:
                pass

    def decompose_MCD(self, x):
        x = self.MCD(x)
        return x  # (B, N, K, T)

    def reconstruct(self, x_imf):  # (B, N, K, T)
        select_feature = self.branch_slelect(x_imf)  # (B, N, K, 2)
        hard_class = gumbel_softmax(select_feature, hard=False)
        x_imf = x_imf.permute(0, 1, 3, 2)  # (B, N, T, K)
        x_summed = torch.matmul(x_imf, hard_class)  # (B, N, T, 2)

        x_left = self.reconstruct_proj_left(
            x_summed[:, :, :, 0])  # (B, N, T) --> # (B, N, T)
        x_right = self.reconstruct_proj_right(x_summed[:, :, :, 1])

        return x_left, x_right

    def forecast(self, total_imf):
        for i in range(2 ** self.total_level * self.K_IMP):
            y_current_imf = self.forecast_proj2[i](
                self.activate(
                    self.forecast_proj1[i](total_imf[:, :, i, :])
                )
            )
            y_current_imf = y_current_imf.unsqueeze(2)
            if i == 0:
                y_imf = y_current_imf
            else:
                y_imf = torch.cat((y_imf, y_current_imf), axis=2)

        return y_imf  # (B, N, 2^level*K, T=input_len)

    def forward(self, x):
        if self.current_level == 0:
            x_imf = self.decompose_MCD(x)
            return x_imf  # (B, N, K, T)
        else:
            x_imf = self.decompose_MCD(x)
            x_left, x_right = self.reconstruct(x_imf)
            imf_left = self.EMDNet_Tree_left(x_left)
            imf_right = self.EMDNet_Tree_left(x_right)
            total_imf = torch.cat([imf_left, imf_right], dim=2)

            if self.current_level == self.total_level:
                y = self.forecast(total_imf)
                return y
            else:
                return total_imf


class IFNet(nn.Module):
    def __init__(self, output_len, input_len, input_dim=18, dec_hidden=18, num_heads=1):
        super(IFNet, self).__init__()
        self.multi_att = MultiHeadSelfAttention(
            dim_in=input_len, dim_k=dec_hidden * num_heads, dim_v=dec_hidden * num_heads, num_heads=num_heads)
        self.predict = nn.Sequential(
            nn.Linear(dec_hidden * num_heads, input_len),
            nn.BatchNorm1d(input_dim, affine=True),  # This line uses input_dim
            nn.LeakyReLU(),
            nn.Linear(input_len, output_len),
        )

    def forward(self, x):
        x = self.multi_att(x)
        x = torch.sum(x, 2)
        x = self.predict(x)
        return x


class DPAD(nn.Module):
    def __init__(self, input_dim, input_len, num_levels, dropout, enc_hidden, dec_hidden, output_len, K_IMP, num_heads):
        super().__init__()
        self.num_levels = num_levels
        self.K_IMP = K_IMP
        self.D_R_D = D_R(
            input_dim=input_dim,
            input_len=input_len,
            total_level=num_levels - 1,
            current_level=num_levels - 1,
            dropout=dropout,
            enc_hidden=enc_hidden,
            output_len=output_len,
            K_IMP=K_IMP
        )
        self.IF = IFNet(
            output_len=output_len,
            input_len=input_len,
            input_dim=input_dim,
            dec_hidden=dec_hidden,
            num_heads=num_heads
        )

    def forward(self, x):
        x = self.D_R_D(x)  # (B, N, 2^(level-1)*K, T=input_len)
        x = self.IF(x)
        return x


class DPAD_ATT(nn.Module):
    def __init__(self, output_len, input_len, input_dim=18, enc_hidden=18, dec_hidden=18, num_levels=3, dropout=0.5,
                 K_IMP=6, RIN=0, num_heads=1):
        super(DPAD_ATT, self).__init__()

        self.input_dim = input_dim
        self.input_len = input_len
        self.output_len = output_len
        self.enc_hidden = enc_hidden
        self.dec_hidden = dec_hidden
        self.num_levels = num_levels
        self.dropout = dropout
        self.K_IMP = K_IMP
        self.RIN = RIN

        print(f"Initializing DPAD with input_dim: {input_dim}, input_len: {input_len}")

        self.DPAD = DPAD(
            input_dim=self.input_dim,
            input_len=self.input_len,
            num_levels=self.num_levels,
            dropout=self.dropout,
            enc_hidden=self.enc_hidden,
            dec_hidden=self.dec_hidden,
            output_len=output_len,  # This should match your target sequence length
            K_IMP=self.K_IMP,
            num_heads=num_heads
        )

        if self.RIN:
            self.affine_weight = nn.Parameter(torch.ones(1, 1, input_dim))
            self.affine_bias = nn.Parameter(torch.zeros(1, 1, input_dim))

        self.final_linear = nn.Linear(input_dim, 1)

    def forward(self, x):
        print(f"Input shape: {x.shape}")

        if self.RIN:
            means = x.mean(1, keepdim=True).detach()
            x = x - means
            stdev = torch.sqrt(
                torch.var(x, dim=1, keepdim=True, unbiased=False) + 1e-5)
            x /= stdev
            x = x * self.affine_weight + self.affine_bias

        x = x.permute(0, 2, 1)  # (B, N, T)
        print(f"Shape after permute: {x.shape}")
        x = self.DPAD(x)
        print(f"Shape after DPAD: {x.shape}")
        x = x.permute(0, 2, 1)  # (B, T, N)
        print(f"Final output shape: {x.shape}")
        x = self.final_linear(x)

        if self.RIN:
            x = x - self.affine_bias
            x = x / (self.affine_weight + 1e-10)
            x = x * stdev
            x = x + means

        return x